# Advanced Modeling Features

## TRISO Particles

OpenMC includes a few convenience functions for generations TRISO particle locations and placing them in a lattice. To be clear, this capability is not a stochastic geometry capability like included in MCNP. It's also important to note that OpenMC does not use delta tracking, which would normally speed up calculations in geometries with tons of surfaces and cells. However, the computational burden can be eased by placing TRISO particles in a lattice.

In [ ]:
%matplotlib inline
from math import pi
import numpy as np
import matplotlib.pyplot as plt
import openmc
import openmc.model

import sys
sys.path.append('../..')
from inputs import *

Let's first start by creating materials that will be used in our TRISO particles and the background material.

In [ ]:
fuel = openmc.Material(name='Fuel')
fuel.set_density('g/cm3', 10.5)
fuel.add_nuclide('U235', 4.6716e-02)
fuel.add_nuclide('U238', 2.8697e-01)
fuel.add_nuclide('O16',  5.0000e-01)
fuel.add_element('C', 1.6667e-01)

buff = openmc.Material(name='Buffer')
buff.set_density('g/cm3', 1.0)
buff.add_element('C', 1.0)
buff.add_s_alpha_beta('c_Graphite')

PyC1 = openmc.Material(name='PyC1')
PyC1.set_density('g/cm3', 1.9)
PyC1.add_element('C', 1.0)
PyC1.add_s_alpha_beta('c_Graphite')

PyC2 = openmc.Material(name='PyC2')
PyC2.set_density('g/cm3', 1.87)
PyC2.add_element('C', 1.0)
PyC2.add_s_alpha_beta('c_Graphite')

SiC = openmc.Material(name='SiC')
SiC.set_density('g/cm3', 3.2)
SiC.add_element('C', 0.5)
SiC.add_element('Si', 0.5)

graphite = openmc.Material()
graphite.set_density('g/cm3', 1.1995)
graphite.add_element('C', 1.0)
graphite.add_s_alpha_beta('c_Graphite')

To actually create individual TRISO particles, we first need to create a universe that will be used within each particle. The reason we use the same universe for each TRISO particle is to reduce the total number of cells/surfaces needed which can improve performance by a factor of two over using unique cells/surfaces in each.

In [ ]:
# Create TRISO universe
spheres = [openmc.Sphere(R=r*1e-4)
           for r in [215., 315., 350., 385.]]
cells = [openmc.Cell(fill=fuel, region=-spheres[0]),
         openmc.Cell(fill=buff, region=+spheres[0] & -spheres[1]),
         openmc.Cell(fill=PyC1, region=+spheres[1] & -spheres[2]),
         openmc.Cell(fill=SiC, region=+spheres[2] & -spheres[3]),
         openmc.Cell(fill=PyC2, region=+spheres[3])]
triso_univ = openmc.Universe(cells=cells)

Now that we have a universe that can be used for each TRISO particle, we need to randomly select locations. In this example, we will select locations at random within in a 1 cm x 1 cm x 1 cm box centered at the origin with a packing fraction of 30%. Note that `pack_trisos` can handle up to the theoretical maximum of 60% (it will just be slow).

In [ ]:
outer_radius = 425.*1e-4

# Create TRISOS here...

## Tally Triggers

Now that we have a cool TRISO model, our boss has asked us what the fission rate in fuel is and he wants to know it such within a relative error (std dev/mean) of 0.1%. Thankfully, OpenMC has a feature called "tally triggers" that allow us to run a simulation until a criterion like this is met, so we don't have to guess how many batches to run.

## External Source Distributions

Thus far, we've only specified `Box` and `Point` spatial sources. OpenMC also allows you to specify the angular and energy distribution of an external source through the `angle` and `energy` attributes of `Source`. If you don't specify an angular distribution, it is isotropic by default. If you don't specify an energy distribution, a `Watt` fission spectrum is assumed.

## Filters and Scores

We've seen a few filter and score types thus far, but there are many more we haven't touched. In addition to specifying domains in which the tallies should be accumulated (cells, materials, universes), we can also specify that only particles within certain ranges of energy score to the tally (`EnergyFilter`) or particles exiting collisions with certain energys (`EnergyoutFilter`). Let's show a quick example of constructing a scattering matrix.

## Fixed Source and Subcritical

OpenMC has primarily been used for eigenvalue simulations, but it does have a fixed source simulation mode. To use this mode, one simply has to change the `Settings.run_mode` attribute. Let's create a simple model with a point source and run it in fixed source mode.